<a href="https://colab.research.google.com/github/camilaandreasbs/Laboratorio-DS/blob/master/Laboratorio9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Camila Bergasa
- Nombre de alumno 2: Sebastián García


### **Link de repositorio de GitHub:** `http://....`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
import plotly.graph_objects as go
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

 Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

### 1. Estudio de la Función

In [2]:
# Se aplica una función que recibe el nombre del subreddit, en este caso Chile, y n_hot = 1000.
# La clase Reddit proporciona un acceso conveniente a la API de Reddit. 
# Le entregamos los datos de un usuario creado por el equipo docente.
# Las instancias de esta clase son la puerta de entrada para interactuar con la API de Reddit a través de la librería PRAW. 
# La forma canónica de obtener una instancia de esta clase es a través de entregarle la id de cliente, una contraseña, un nombre de agente y un nombre de usuario.
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

# Define 3 diccionarios vacíos votes, post, url
# Genera una lista de los post mas votados, limitados por el n_hot definido en la función
# Se recorre la lista generada anteriormente, y se van a entregar los votos arriba, 
# los votos abajo, la url y el post, guardandolos en 4 listas vacías diferentes.

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

# Se recorre la lista generada en el for anterior,
# Se genera un ratio generado por la librería para definir ups
# Se va agregando a cada lista 

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                # Va agregando los resultados a las listas upvotes (Agrega la calificacion ups como tal), downvotes (agrega la calificación menos el score de submission), post (agrega el título) y url (agrega la url)
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        #Finalmente genera un vector o un arreglo con las listas upvotes y downvotes, para finalmente retornar este arreglo junto con post y url
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


#### 2. Tiempos de Ejecución

In [3]:
# Se ocupa el módulo timeit para medir el tiempo de ejecución de la función praw_reddit
# timeit ejecuta por defecto 10 millones de veces el código, repite 7 veces 
# el experimento y reporta el tiempo de ejecución promedio.
%timeit praw_reddit

The slowest run took 42.24 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 38.2 ns per loop


In [4]:
votes, post, url = praw_reddit()

In [5]:
# Tiempo que demora en ejecutarse 10 millones de veces.
%timeit praw_reddit

The slowest run took 33.13 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 38.5 ns per loop


In [6]:
# Usamos el perfilador prun para obtener cuánto se demora la ejecución de cada 
# una de las operaciones que estamos ejecutando. Indica cuanto se demoró
# cada una de las instrucciones que se ejecutaron en praw_reddit
%prun praw_reddit

In [7]:
%prun votes, post, url = praw_reddit()

Se aprecia que la operación más pesada es filename:lineno(function) con  aproximadamente 11.164 segundos de ejecución

In [8]:
# Usamos lineprofiler para perfilar por línea de código
%load_ext line_profiler

In [11]:
%lprun -f praw_reddit praw_reddit(nombre_subreddit="chile", n_hot=1000)

3. Estudio de la Memoria ocupada por el código

In [12]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.59.0-py3-none-any.whl size=31313 sha256=aaa821fbd5e20b50d55e375668ce9094a0174e31a1596b56a44718b001d43fad
  Stored in directory: /root/.cache/pip/wheels/8d/6e/d2/af9dae73f8fef0c64c18b0a02a69fbd4c65b854912fa87a390
Successfully built memory-profiler


In [13]:
%load_ext memory_profiler

In [14]:
%memit praw_reddit(nombre_subreddit="chile", n_hot=1000)

peak memory: 242.21 MiB, increment: 0.02 MiB


3.1 Estudio de la memoria ocupada línea por línea

In [15]:
%%file praw_module.py
import praw
from praw import Reddit
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url



Writing praw_module.py


In [16]:
from praw_module import praw_reddit

%mprun -f praw_reddit praw_reddit()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 845, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/memory_profiler.py", line 848, in disable
    sys.settrace(self._original_trace_function)



4.  ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?
**Respuesta:**

Al usar lprun se perfila por línea de código y se obtiene que la línea que toma más tiempo de ejecución corresponde a la línea 24 (top_submissions = list(subreddit.hot(limit=n_hot))), esto se debe a que en ese momento define a los top submissions como una lista de los subbredits destacados cuyo límite n_hot es de 1000.

Para el caso de la memoria , al ejecutarse la línea mágica %memit sobre la función praw_reddit, se obtiene lo siguiente: peak memory: 155.73 MiB, increment: 6.10 MiB. Lo cuál significa que ejecutar la función tomó 155.73 megas de memoria RAM en su peak existiendo un incremento de 6.10 MiB. Si se realiza el análisis línea por línea se aprecia que la cantidad de megas utilizados es de aproximadamente 151.7 MiB hasta que llega a la línea top_submissions = list(subreddit.hot(limit=n_hot)), en donde se da un incremento de 7.2 MiB, esto es importante ya que esa línea queda almacenada en la memoria y luego el uso de memoria es de 151.7 + 7.2 = 158.9 hasta el final de la función. 

Esta línea ocupa más tiempo y memoria debido a que como se mencionó anteriormente, crea la instancia de subreddit y enlista los posts más hot para generar los diccionarios y es probable que esto implique leer mucha información.


## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**


¿Qué es la memoria cache y a que se refiere las siglas LRU?
>La memoria cache corresponde a un espacio de memoria en la cuál se almacenan elementos temporalmente y habitualmente para un único uso. Es una memoria que funciona de forma similar a la memoria principal de una computadora, pero es de menor tamaño y de acceso más rápido. Es una memoria rápida y eficiente usada para reducir el tiempo de acceso a datos ubicados en la memoria principal.
Las siglas LRU significan Last Recently Used (Menos usado recientemente), lo cuál hace referencia a un tipo de algoritmo que realiza un descarte de los elementos menos usados recientemente. Los algoritmos de caché, son programas que optimizan la información en la memoria caché de un computador. Cuando se llena, el algoritmo elige a los elementos menos usados recientemente para liberar espacio y añadir nuevos elementos.

¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
> Los costos que tienen la aplicación de técnicas de Caching dicen relación con el hecho de que aumenta el consumo de memoria, si la memoria se encuentra localizada en el disco acceder puede ser lento y el rendimiento puede decaer, por lo que antes de aplicar estas técnicas es recomendable estudiar su factibilidad, considerando las políticas de almacenamiento y de acceso a los resultados y cómo se relacionan con el rendimiento del programa a implementar.

¿Cuál es la consecuencía de ocupar caching en la función anterior?
> La consecuencia de aplicar caching en la función anterior es que se ejecutará la función y se van a guardar los resultados de la función praw_reddit y no deberán ser calculados nuevamente ya que el cálculo de la operación puede ser muy costoso. En resumen, al memoizar praw_reddit() se llama en múltiples ocasiones a la función sin tener que incurrir en el costo de la línea subreddit.hot(limit=n_hot).

In [17]:
###### Código Aquí ######
# Mejora del código con caché
from functools import lru_cache


Se mide el tiempo de ejecución antes, para n_hot = 1000 y se obtiene un tiempo de ejecución de 13.9 s per loop

In [18]:
%%timeit
praw_reddit(nombre_subreddit="chile", n_hot=1000)

1 loop, best of 5: 12.1 s per loop


Se memoíza la función y se repite el experimento de la celda anterior

In [19]:
@lru_cache(maxsize = 1000)
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []
        

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


In [20]:
%%timeit
praw_reddit(nombre_subreddit="chile", n_hot=1000)

The slowest run took 31735323.86 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 403 ns per loop


Se comprueba la eficiencia del método, obteniendo un tiempo de ejecución de 371 ns per loop


**Respuesta de lru_cache sobre praw_reddit**

`Las consecuencias de utilizar LRU sobre la función praw_reddit es una disminución en su tiempo de ejecución, de 13.9 segundos a 371 nanosegundos. Por otro lado el tiempo de ejecución de la celda también se redujo. Al aplicar LRU sobre praw_reddit se guardaron los resultados para los argumentos seleccionados, por lo que estos ya no deben ser calculados nuevamente. Esto se traduce en una disminución del tiempo de ejecución.`

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

Función sin uso de Numpy

In [21]:
votes, post, url = praw_reddit()

In [22]:
def intervals(votes):
    sigma1 = []
    mu1 = []
    for d,u in votes:
        a = 1 + u
        b = 1 + d
        mu = a/(a+b)
        mu1.append(mu)
        sigma = 1.65*((a*b/((a+b+1)*(a+b)**2))**(0.5))
        sigma1.append(sigma)
    return mu1, sigma1

Función con uso de Numpy

In [35]:
def intervals_numpy(votes):
    a = votes[0] + 1
    b = votes[1] + 1
    c = np.multiply(a,b)
    d = ((a+b+1)*(a+b)**2)**(0.5)
    sigma = 1.65*(np.sqrt(c/d))
    mu = a/(a+b)
    return mu, sigma

## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Ahora, grafiquen los tiempos tomados y comente los desempeños.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

In [36]:
###### Código Aquí ######
# Generación de los batches de datos que contiene el diccionario votes
def mu_time(python, numpy):
    python_times = np.empty([1000,len(votes)])
    np_times = np.empty([1000,len(votes)])
    for i in range(1000):
        for n in range(len(votes)):
            t = time.process_time()
            python(votes[n]);
            python_times[i,n]=time.process_time()-t
            t = time.process_time()
            numpy(votes[n]);
            np_times[i,n] = time.process_time()-t
    return python_times, np_times

python_times, np_times = mu_time(intervals, intervals_numpy)
python_time = [np.mean(python_times[:,i]) for i in range(len(votes))]
numpy_time = [np.mean(np_times[:,i]) for i in range(len(votes))]

def comparative(python_time, numpy_time):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
                    x = [50 + i*50 for i in range(len(votes))],
                    y = python_time,
                    name='Python'))
    fig.add_trace(go.Scatter(
                    x = [50 + i*50 for i in range(len(votes))],
                    y = numpy_time,
                    name='Numpy'))

    fig.show()
    return
comparative(python_time, numpy_time)

¿Es posible observar una diferencia? ¿a qué se deberá esto?.
 > Si es posible apreciar una diferencia, para el caso de Numpy los tiempos se mantienen constantes a medida que se incrementa la cantidad de elementos en el diccionario. Mientras que para el caso de la función creada con iteradores de Python, los tiempos aumentan a medida que se incrementa la cantidad de elementos en el diccionario. Esto puede deberse a que usando los iteradores de python se ejecutan fors respecto de la cantidad de posts y en numpy se realiza una cantidad constante de operaciones que son vectoriales y que no dependen del número de posts considerados.

In [31]:
# Aplicación de compilador NUMBA

@jit(nopython = True)
def jit_intervals(votes):
    sigma1 = []
    mu1 = []
    for d,u in votes:
        a = 1 + u
        b = 1 + d
        mu = a/(a+b)
        mu1.append(mu)
        sigma = 1.65*((a*b/((a+b+1)*(a+b)**2))**(0.5))
        sigma1.append(sigma)
    return mu1, sigma1


@jit(nopython = True)
def jit_intervals_numpy(votes):
  a = votes[0] + 1
  b = votes[1] + 1
  c = a*b
  d = ((a+b+1)*(a+b)**2)**(0.5)
  sigma = 1.65*(np.sqrt(c/d))
  mu = a/(a+b)
  return mu, sigma


Luego de aplicar el compilador, se grafican nuevamente los resultados. Se aprecia que si se ejecuta varias veces el gráfico se observan resultados direfentes (por eso los pusimos tres veces). En el primero, en ambos disminuye el tiempo de ejecución drásticamente y luego el de python aumenta de manera lineal igual que antes, pero con una pendiente mucho más suave mientras que para el codigo realizado con numpy se mantiene constante y muy cercano a cero. Sin embargo, al correr de nuevo el código, vuelve a adquirir un comportamiento similar al anterior.

In [32]:
python_times, np_times = mu_time(jit_intervals, jit_intervals_numpy)
jit_python = [np.mean(python_times[:,i]) for i in range(len(votes))]
jit_numpy = [np.mean(np_times[:,i]) for i in range(len(votes))]
comparative(jit_python, jit_numpy)

In [33]:
python_times, np_times = mu_time(jit_intervals, jit_intervals_numpy)
jit_python = [np.mean(python_times[:,i]) for i in range(len(votes))]
jit_numpy = [np.mean(np_times[:,i]) for i in range(len(votes))]
comparative(jit_python, jit_numpy)

In [34]:
python_times, np_times = mu_time(jit_intervals, jit_intervals_numpy)
jit_python = [np.mean(python_times[:,i]) for i in range(len(votes))]
jit_numpy = [np.mean(np_times[:,i]) for i in range(len(votes))]
comparative(jit_python, jit_numpy)

grafiquen los tiempos tomados y comente los desempeños.

## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [37]:
###### Código Aquí ######
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervals_numpy(votos[:, 0], votos[:, 1])
posterior_mean, std_err = intervals_numpy(votes)
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

lower bounds aproximados:


TypeError: ignored

**Respuesta:**

`Escriba su respuesta aquí`

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>